# Adverserial Variational Optimization

*Gilles Louppe & Kayle Cranmer*

Notebook by Joeri Hermans

In [1]:
!date

Tue Oct 10 16:01:31 CEST 2017


Complex computer simulators are increasingly used across fields of science as generative models tying parameters of an underlying theory to experimental observations. Inference in this setup is often difficult, as simulators rarely admit a tractable density or likelihood function. We introduce Adversarial Variational Optimization (AVO), a likelihood-free inference algorithm for fitting a non-differentiable generative model incorporating ideas from empirical Bayes and variational inference. We adapt the training procedure of generative adversarial networks by replacing the differentiable generative network with a domain-specific simulator. We solve the resulting non-differentiable minimax problem by minimizing variational upper bounds of the two adversarial objectives. Effectively, the procedure results in learning a proposal distribution over simulator parameters, such that the corresponding marginal distribution of the generated data matches the observations. We present results of the method with simulators producing both discrete and continuous data.

**Manuscript**: [https://arxiv.org/abs/1707.07113](https://arxiv.org/abs/1707.07113)

## Configuration

### Imports

In [4]:
import numpy as np
import torch

from sklearn.utils import check_random_state

### Globals

In [5]:
seed = 1337
random_number_generator = check_random_state(seed)
batch_size = 64
num_epochs = 300
lambda_gp = 0.0025
theta_true = np.array([(42.0-40)/(50-40), (0.9 - 0.5) / (1.5-0.5)])

## Introduction

## Conclusion